In [4]:
! git clone https://github.com/AryaTopale/NLP_task.git


fatal: destination path 'NLP_task' already exists and is not an empty directory.


fatal: not a git repository (or any of the parent directories): .git


In [ ]:
!pip install nltk

In [ ]:
!pip install tensorflow

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Lambda
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer


# Stopwords

In [ ]:
import numpy as np
import nltk
import pandas as pd
import re

In [ ]:
nltk.download('stopwords')
nltk.download('reuters')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('webtext')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
from nltk.corpus import reuters
from nltk.corpus import webtext
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package webtext to /root/nltk_data...
[nltk_data]   Unzipping corpora/webtext.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Preprocessing

In [ ]:
def preprocessing(content):
    # Tokenization
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(content)
    sequences = tokenizer.texts_to_sequences(content)

    # Convert sequences back to words
    word_index = tokenizer.index_word
    tokens = [word_index[token] for seq in sequences for token in seq if token in word_index]

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in filtered_tokens]

    return lemmatized_tokens

### Preprocessing Steps

1. **Lemmatization**  
   Converts words to their base forms based on context and POS (e.g., *"running"* → *"run"*, *"better"* → *"good"*).

2. **Tokenization**  
   Splits text into smaller units (tokens), like words or symbols (e.g., *"Hi, I am Arya!"* → `["Hi", ",","I", "am", "Arya","!]`).

3. **Stop Word Removal**  
   Removes common words (e.g., *"and"*, *"the"*) to focus on meaningful content (e.g., *"The cat is black."* → *"cat black"*).


Getting the Corpus- I will be using nltk's brown corpus with different categories such as news,fiction,government etc

In [ ]:
nltk.download('brown')
from nltk.corpus import brown

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


In [ ]:
# Prepare the corpus text for processing
corpus_text = brown.sents()  # Use sentences for Tokenizer
processed_tokens = preprocessing(corpus_text)
print(processed_tokens[:100])

['fulton', 'county', 'grand', 'jury', 'said', 'friday', 'investigation', "atlanta's", 'recent', 'primary', 'election', 'produced', '``', 'evidence', "''", 'irregularity', 'took', 'place', '.', 'jury', 'said', 'term-end', 'presentment', 'city', 'executive', 'committee', ',', 'over-all', 'charge', 'election', ',', '``', 'deserves', 'praise', 'thanks', 'city', 'atlanta', "''", 'manner', 'election', 'conducted', '.', 'september-october', 'term', 'jury', 'charged', 'fulton', 'superior', 'court', 'judge', 'durwood', 'pye', 'investigate', 'report', 'possible', '``', 'irregularity', "''", 'hard-fought', 'primary', 'mayor-nominate', 'ivan', 'allen', 'jr.', '.', '``', 'relative', 'handful', 'report', 'received', "''", ',', 'jury', 'said', ',', '``', 'considering', 'widespread', 'interest', 'election', ',', 'number', 'voter', 'size', 'city', "''", '.', 'jury', 'said', 'find', 'many', "georgia's", 'registration', 'election', 'law', '``', 'outmoded', 'inadequate', 'often', 'ambiguous']


# Making Vocabulary

In [ ]:
vocab = list(set(processed_tokens))
word_to_index = {word: i for i, word in enumerate(vocab)}
index_to_word = {i: word for word, i in word_to_index.items()}
vocab_size = len(vocab)
print(vocab_size)

44410


Generating context target pairs

In [ ]:
# Configuration
embedding_size = 300
window_size = 2
vocab_size = len(word_to_index)

# Generate context-target pairs
def generate_context_target_pairs(tokens, window_size):
    contexts, targets = [], []
    for i in range(window_size, len(tokens) - window_size):
        context = (
            tokens[i - window_size:i] + tokens[i + 1:i + window_size + 1]
        )
        target = tokens[i]
        contexts.append([word_to_index[word] for word in context])
        targets.append(word_to_index[target])
    return np.array(contexts), np.array(targets)

# Generate sample pairs
contexts, targets = generate_context_target_pairs(processed_tokens, window_size)

Continuous Bag of Words Neural Network

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Lambda, Dense
from tensorflow.keras.optimizers import Adam

In [ ]:
context_input = Input(shape=(window_size * 2,), dtype='int32')
context_embedding = Embedding(vocab_size, embedding_size, input_length=window_size * 2)(context_input)
context_mean = Lambda(lambda x: K.mean(x, axis=1))(context_embedding)
output = Dense(vocab_size, activation='softmax')(context_mean)
model = Model(inputs=context_input, outputs=output)
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 4)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 4, 300)              │      13,323,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lambda (Lambda)                      │ (None, 300)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 44410)               │      13,367,410 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 26,690,410 (101.82 MB)

 Trainable params: 26,690,410 (101.82 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
X_train=contexts
Y_train=targets
Y_train = Y_train.reshape(-1, 1)

In [ ]:
epochs = 10 # Set the number of epochs
batch_size = 64  # Set batch size

model.fit(X_train,Y_train, epochs=epochs, batch_size=batch_size)

Epoch 1/5
 9201/10722 ━━━━━━━━━━━━━━━━━━━━ 14:52 587ms/step - loss: 8.3911